In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Historic Generation Data

In [2]:
gen_data = pd.read_csv("df_fuel_ckan.csv")
#historic generation data 

In [3]:
gen_data.head()

,DATETIME,GAS,COAL,NUCLEAR,WIND,HYDRO,IMPORTS,BIOMASS,OTHER,SOLAR,...,IMPORTS_perc,BIOMASS_perc,OTHER_perc,SOLAR_perc,STORAGE_perc,GENERATION_perc,LOW_CARBON_perc,ZERO_CARBON_perc,RENEWABLE_perc,FOSSIL_perc
0,2009-01-01 00:00:00+00,8396.0,15038.0,7098.0,273.0,246.0,2506.0,0.0,0.0,0.0,...,7.5,0.0,0.0,0.0,0.0,100.0,22.7,22.7,1.5,69.8
1,2009-01-01 00:30:00+00,8526.0,15097.0,7086.0,253.0,245.0,2485.0,0.0,0.0,0.0,...,7.4,0.0,0.0,0.0,0.0,100.0,22.5,22.5,1.5,70.1
2,2009-01-01 01:00:00+00,8492.0,15083.0,7073.0,228.0,246.0,2450.0,0.0,0.0,0.0,...,7.3,0.0,0.0,0.0,0.0,100.0,22.5,22.5,1.4,70.2
3,2009-01-01 01:30:00+00,8335.0,15015.0,7063.0,212.0,246.0,2420.0,0.0,0.0,0.0,...,7.3,0.0,0.0,0.0,0.0,100.0,22.6,22.6,1.4,70.1
4,2009-01-01 02:00:00+00,8304.0,14975.0,7051.0,196.0,246.0,2349.0,0.0,0.0,0.0,...,7.1,0.0,0.0,0.0,0.0,100.0,22.6,22.6,1.3,70.3


In [4]:
gen_data['DATETIME'] = pd.to_datetime(gen_data['DATETIME'])

In [5]:

# Remove the timezone information from the datetime column
gen_data['DATETIME'] = gen_data['DATETIME'].dt.tz_localize(None)



In [6]:
gen_data.columns

Index(['DATETIME', 'GAS', 'COAL', 'NUCLEAR', 'WIND', 'HYDRO', 'IMPORTS',
       'BIOMASS', 'OTHER', 'SOLAR', 'STORAGE', 'GENERATION',
       'CARBON_INTENSITY', 'LOW_CARBON', 'ZERO_CARBON', 'RENEWABLE', 'FOSSIL',
       'GAS_perc', 'COAL_perc', 'NUCLEAR_perc', 'WIND_perc', 'HYDRO_perc',
       'IMPORTS_perc', 'BIOMASS_perc', 'OTHER_perc', 'SOLAR_perc',
       'STORAGE_perc', 'GENERATION_perc', 'LOW_CARBON_perc',
       'ZERO_CARBON_perc', 'RENEWABLE_perc', 'FOSSIL_perc'],
      dtype='object')

# Market Index Price Data 

In [7]:
d_2014 = pd.read_csv("MID_2014.csv",encoding='latin')
d_2015 = pd.read_csv("MID_2015.csv",encoding='latin')
d_2016 = pd.read_csv("MID_2016.csv",encoding='latin')
d_2017= pd.read_csv("MID_2017.csv",encoding='latin')
d_2018= pd.read_csv("MID_2018.csv",encoding='latin')
d_2019= pd.read_csv("MID_2019.csv",encoding='latin')
d_2020 = pd.read_csv("MID_2020.csv",encoding='latin')
d_2021 = pd.read_csv("MID_2021.csv",encoding='latin')
d_2022 = pd.read_csv("MID_2022.csv",encoding='latin')
d_2023 = pd.read_csv("MID_2023.csv",encoding='latin')

In [8]:
market_index_data = pd.concat([d_2014,d_2015,d_2016,d_2017,d_2018,d_2019,d_2020,d_2021,d_2022,d_2023],
                    axis=0,ignore_index= True)

In [9]:
#creating a new column with DATETIME 
import datetime
from datetime import datetime
market_index_data['date'] = market_index_data['Settlement Date'].apply(lambda x: datetime.strptime(x, '%d %B %Y'))
market_index_data['Time Interval'] = (market_index_data['Settlement Period'] - 1) * 30

market_index_data['date'] = market_index_data['date'].dt.strftime('%Y-%m-%d')
import datetime
market_index_data['Time Format'] = market_index_data['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())


In [10]:
market_index_data['Time Format'] = market_index_data['Time Format'].apply(lambda x: x.strftime('%H:%M:%S'))
market_index_data['DATETIME'] = pd.to_datetime(market_index_data['date'] + ' ' + market_index_data['Time Format'])
market_index_data['Time Format'] = market_index_data['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())


In [11]:
#market_index_data = market_index_data.drop('DateTime',axis =1)

In [12]:
mid_1 = market_index_data.groupby('DATETIME').agg({'Settlement Period': 'first',
                                                   'Market Index Price (£/MWh)': 'sum', 
                                                   'Market Index Volume (MWh)': 'sum'}).reset_index()


def custom_agg(s):
    non_zero_values = s.loc[s != 0]
    if non_zero_values.empty:
        return 0
    else:
        return non_zero_values.iloc[0]


In [13]:
mid_1.head(10)

,DATETIME,Settlement Period,Market Index Price (£/MWh),Market Index Volume (MWh)
0,2014-01-01 00:00:00,1,35.32,301.50
1,2014-01-01 00:30:00,2,37.28,365.65
2,2014-01-01 01:00:00,3,41.50,184.00
3,2014-01-01 01:30:00,4,36.74,185.15
4,2014-01-01 02:00:00,5,32.04,283.50
5,2014-01-01 02:30:00,6,30.51,322.50
6,2014-01-01 03:00:00,7,26.62,454.30
7,2014-01-01 03:30:00,8,24.41,566.95
8,2014-01-01 04:00:00,9,22.39,554.40
9,2014-01-01 04:30:00,10,20.75,666.65


In [14]:
mask = (market_index_data['Market Index Data Provider Id'] == 'N2EXMIDP') & (market_index_data['Market Index Volume (MWh)'] != 0)


In [15]:
market_index_data[mask];

In [16]:
market_index_data['Market Index Data Provider Id'].value_counts()

N2EXMIDP    166030
APXMIDP     165982
Name: Market Index Data Provider Id, dtype: int64

we can see the difference nordic provider sometimes provided in other settlement periods

In [17]:
market_index_data.tail(10)

,Settlement Date,Settlement Period,Market Index Data Provider Id,Market Index Volume (MWh),Market Index Price (£/MWh),date,Time Interval,Time Format,DATETIME
332002,21 June 2023,44,APXMIDP,1353.50,131.81,2023-06-21,1290,21:30:00,2023-06-21 21:30:00
332003,21 June 2023,44,N2EXMIDP,0.00,0.00,2023-06-21,1290,21:30:00,2023-06-21 21:30:00
332004,21 June 2023,45,APXMIDP,1411.20,127.15,2023-06-21,1320,22:00:00,2023-06-21 22:00:00
332005,21 June 2023,45,N2EXMIDP,0.00,0.00,2023-06-21,1320,22:00:00,2023-06-21 22:00:00
332006,21 June 2023,46,APXMIDP,1544.65,110.64,2023-06-21,1350,22:30:00,2023-06-21 22:30:00
332007,21 June 2023,46,N2EXMIDP,0.00,0.00,2023-06-21,1350,22:30:00,2023-06-21 22:30:00
332008,21 June 2023,47,APXMIDP,899.35,116.71,2023-06-21,1380,23:00:00,2023-06-21 23:00:00
332009,21 June 2023,47,N2EXMIDP,0.00,0.00,2023-06-21,1380,23:00:00,2023-06-21 23:00:00
332010,21 June 2023,48,APXMIDP,664.35,102.45,2023-06-21,1410,23:30:00,2023-06-21 23:30:00
332011,21 June 2023,48,N2EXMIDP,0.00,0.00,2023-06-21,1410,23:30:00,2023-06-21 23:30:00


# Historic Demand Data

In [18]:
import pandas as pd
import glob


# Define the selected filenames or patterns
selected_files = ['demanddata_2009.csv', 'demanddata_2010.csv', 'demanddata_2011.csv','demanddata_2012.csv',
                  'demanddata_2013.csv','demanddata_2014.csv','demanddata_2015.csv','demanddata_2016.csv'
                  ,'demanddata_2017.csv','demanddata_2018.csv','demanddata_2019.csv','demanddata_2020.csv',
                 'demanddata_2021.csv','demanddata(1).csv','demanddata.csv']


In [19]:

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through the selected files
for file_pattern in selected_files:
    df = pd.read_csv(file_pattern)
    dataframes.append(df)

# Concatenate all the DataFrames into a single DataFrame
Demand_Data = pd.concat(dataframes, ignore_index=True)




In [20]:
Demand_Data.head()

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW
0,01-JAN-2009,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0,NaN,NaN
1,01-JAN-2009,2,38047,38964,34072,53,1403,0,0,0,157,2002,0,0,-160,0,0,NaN,NaN
2,01-JAN-2009,3,37380,38651,33615,53,1403,0,0,0,511,2002,0,0,-160,0,0,NaN,NaN
3,01-JAN-2009,4,36426,37775,32526,50,1403,0,0,0,589,1772,0,0,-160,0,0,NaN,NaN
4,01-JAN-2009,5,35687,37298,31877,50,1403,0,0,0,851,1753,0,0,-160,0,0,NaN,NaN


In [21]:
Demand_Data.tail()

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW
253577,2023-06-19,44,26074,28088,24393,1055,6538,0,14285,0,5,430,-2,-921,-306,0,-280,1298.0,0.0
253578,2023-06-19,45,25811,27527,24183,1060,6538,0,14285,0,5,292,-2,-789,-322,0,-98,1298.0,10.0
253579,2023-06-19,46,24784,26515,23256,1064,6538,0,14285,0,6,272,-2,-779,-358,0,-86,1274.0,9.0
253580,2023-06-19,47,23080,25660,21736,1051,6538,0,14285,0,4,-476,-2,-806,-316,0,-334,904.0,-142.0
253581,2023-06-19,48,21963,24722,20673,1038,6538,0,14285,0,73,-540,-2,-806,-330,0,-352,825.0,-156.0


In [22]:
import datetime
#Demand_Data['date'] = Demand_Data['SETTLEMENT_DATE'].apply(lambda x: datetime.strptime(x, '%d %B %Y'))
Demand_Data['Time Interval'] = (Demand_Data['SETTLEMENT_PERIOD'] - 1) * 30

#Demand_Data['date'] = Demand_Data['date'].dt.strftime('%Y-%m-%d')

Demand_Data['Time Format'] = Demand_Data['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())

Demand_Data['Time Format'] = Demand_Data['Time Format'].apply(lambda x: x.strftime('%H:%M:%S'))

Demand_Data['DATETIME'] = pd.to_datetime(Demand_Data['SETTLEMENT_DATE'] + ' ' + Demand_Data['Time Format'])




we have 3 data sets now demand data market index price generation data 

# SSP(system sell price) /SBP (system buy price) data - Elexon data 

## System Sell Price

In [133]:
SSP = pd.read_excel('prices.xls','SSP')

In [134]:
SSP.head()

,Date,Run,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,50
0,2001-03-27,DF,2.76676,6.23181,5.92955,0.61020,0.12778,-9.97316,28.28724,38.18177,...,7.90359,0.78551,0.13840,8.45620,9.60637,10.54528,9.81334,10.10231,NaN,NaN
1,2001-03-28,DF,10.00696,10.85647,9.28724,9.04365,4.44915,-100.00000,4.00000,15.25000,...,0.00000,13.35298,6.99595,10.18164,9.24505,18.20735,3.57793,6.16488,NaN,NaN
2,2001-03-29,DF,6.43443,6.89398,7.22250,1.99784,15.25000,-100.00000,15.25000,15.25000,...,-26.56463,-3.20714,-5.36642,-11.88932,-15.80997,-18.28517,1.49349,0.14706,NaN,NaN
3,2001-03-30,DF,3.92383,5.81459,5.93008,-0.10113,-15.30681,-20.25186,-11.01626,15.25000,...,4.36315,-66.18367,-9.43636,-8.25770,-10.15610,-10.47089,-9.16298,-5.02416,NaN,NaN
4,2001-03-31,DF,-12.44055,-10.11949,2.47943,5.99630,7.20452,10.62924,7.19239,5.97222,...,9.35580,8.02308,9.02920,6.97000,9.53125,-8.50344,-27.23651,-26.89920,NaN,NaN


In [135]:


# Add an average column by averaging the rows
#SSP["Average"] = SSP.iloc[:,2:50].mean(axis=1)



In [26]:
#SSP_1 = SSP.iloc[:, SSP.columns != 'Average']

In [136]:
SSP_1 = SSP.iloc[:, SSP.columns != 'Run']

In [138]:
SSP_1.head()

,Date,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2001-03-27,2.76676,6.23181,5.92955,0.61020,0.12778,-9.97316,28.28724,38.18177,24.43990,...,7.90359,0.78551,0.13840,8.45620,9.60637,10.54528,9.81334,10.10231,NaN,NaN
1,2001-03-28,10.00696,10.85647,9.28724,9.04365,4.44915,-100.00000,4.00000,15.25000,15.25000,...,0.00000,13.35298,6.99595,10.18164,9.24505,18.20735,3.57793,6.16488,NaN,NaN
2,2001-03-29,6.43443,6.89398,7.22250,1.99784,15.25000,-100.00000,15.25000,15.25000,15.25000,...,-26.56463,-3.20714,-5.36642,-11.88932,-15.80997,-18.28517,1.49349,0.14706,NaN,NaN
3,2001-03-30,3.92383,5.81459,5.93008,-0.10113,-15.30681,-20.25186,-11.01626,15.25000,9.80000,...,4.36315,-66.18367,-9.43636,-8.25770,-10.15610,-10.47089,-9.16298,-5.02416,NaN,NaN
4,2001-03-31,-12.44055,-10.11949,2.47943,5.99630,7.20452,10.62924,7.19239,5.97222,5.83519,...,9.35580,8.02308,9.02920,6.97000,9.53125,-8.50344,-27.23651,-26.89920,NaN,NaN


In [145]:
sd = SSP_1.melt(id_vars='Date',var_name= 'Settlement_Period',
                value_name='System Sell Price(GBP/MWh)')
sd['Settlement_Period'] = sd['Settlement_Period'].astype('int')
#sort by date
sd = sd.sort_values(by=['Date', 'Settlement_Period']).reset_index(drop=True)


In [150]:
sd = sd.dropna()

In [157]:
sd[sd['Date'] == '2019-01-01'];

In [158]:
sd.columns

Index(['Date', 'Settlement_Period', 'System Sell Price(GBP/MWh)'], dtype='object')

In [165]:
SSP_ = sd.reset_index(drop=True)

In [176]:
SSP_[SSP_['Date'] == '2019-01-01'].head()

,Date,Settlement_Period,System Sell Price(GBP/MWh),Time Interval,Time Format,Timestamp
311474,2019-01-01,1,15.0,0,00:00:00,2019-01-01 00:00:00
311475,2019-01-01,2,15.0,30,00:30:00,2019-01-01 00:30:00
311476,2019-01-01,3,16.0,60,01:00:00,2019-01-01 01:00:00
311477,2019-01-01,4,16.0,90,01:30:00,2019-01-01 01:30:00
311478,2019-01-01,5,16.0,120,02:00:00,2019-01-01 02:00:00


In [168]:
import datetime

SSP_['Time Interval'] = (SSP_['Settlement_Period']-1)*30
#changing settlement periods to time 

In [173]:
SSP_['Time Format'] = SSP_['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())
#changing into minutes

In [36]:
type(SSP_['Date'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [175]:
SSP_['Timestamp'] = SSP_['Date'] + pd.to_timedelta(SSP_['Time Format'].astype(str))


In [177]:
SSP_['Timestamp'] = pd.to_datetime(SSP_['Timestamp'])

In [184]:
SSP__[SSP__['Date'] == '2019-01-01'].head()

,Date,Settlement_Period,System Sell Price(GBP/MWh),DATETIME
311474,2019-01-01,1,15.0,2019-01-01 00:00:00
311475,2019-01-01,2,15.0,2019-01-01 00:30:00
311476,2019-01-01,3,16.0,2019-01-01 01:00:00
311477,2019-01-01,4,16.0,2019-01-01 01:30:00
311478,2019-01-01,5,16.0,2019-01-01 02:00:00


In [181]:
SSP_['DATETIME'] = SSP_['Timestamp'] 

In [182]:
SSP__ = SSP_.drop(['Time Interval','Time Format','Timestamp'],axis=1)

In [185]:
SSP__.head()

,Date,Settlement_Period,System Sell Price(GBP/MWh),DATETIME
0,2001-03-27,1,2.76676,2001-03-27 00:00:00
1,2001-03-27,2,6.23181,2001-03-27 00:30:00
2,2001-03-27,3,5.92955,2001-03-27 01:00:00
3,2001-03-27,4,0.61020,2001-03-27 01:30:00
4,2001-03-27,5,0.12778,2001-03-27 02:00:00


## System Buy Price

In [192]:
SBP = pd.read_excel('prices.xls','SBP')

In [193]:
SBP.head()

,Date,Run,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,50
0,2001-03-27,DF,20.97549,19.58800,22.61021,38.08869,33.67798,39.83499,28.28724,38.18177,...,279.38841,251.53062,217.87948,56.96316,30.74486,30.00000,20.69922,22.50088,NaN,NaN
1,2001-03-28,DF,24.21956,33.00606,38.51463,54.92309,70.95632,57.04822,47.75852,51.41475,...,97.24102,193.65681,32.98831,349.52532,43.25098,36.52121,65.51957,57.17138,NaN,NaN
2,2001-03-29,DF,98.59603,63.34568,32.32387,38.84969,32.39927,53.39075,30.62564,38.26297,...,144.92755,390.60470,662.13826,914.76630,262.19745,185.72044,20.23791,17.29615,NaN,NaN
3,2001-03-30,DF,23.39333,23.72748,27.00277,31.89989,34.58617,33.67641,30.08851,28.58215,...,29.08666,848.86219,311.27570,61.76475,191.99345,110.09488,25.36472,22.76406,NaN,NaN
4,2001-03-31,DF,32.20507,31.17349,32.93606,48.93056,43.35069,44.32983,42.68181,35.60474,...,251.39846,865.83405,853.61869,582.73810,103.94630,79.36612,43.67538,56.03893,NaN,NaN


In [194]:
SBP_1 = SBP.iloc[:, SBP.columns != 'Run']

In [195]:
sb = SBP_1.melt(id_vars='Date',var_name= 'Settlement_Period',
                value_name='System Buy Price(GBP/MWh)')
SBP_ = sb.sort_values(by=['Date', 'Settlement_Period']).reset_index(drop = True)

In [196]:
import datetime

SBP_['Time Interval'] = (SBP_['Settlement_Period']-1)*30

SBP_['Time Format'] = SBP_['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())

SBP_['Timestamp'] = SBP_['Date'] + pd.to_timedelta(SBP_['Time Format'].astype(str))

SBP_['Timestamp'] = pd.to_datetime(SBP_['Timestamp'])

In [205]:
SBP__[SBP__['Date'] == '2019-01-01'].head()

,Date,Settlement_Period,System Buy Price(GBP/MWh),DATETIME
324450,2019-01-01,1,15.0,2019-01-01 00:00:00
324451,2019-01-01,2,15.0,2019-01-01 00:30:00
324452,2019-01-01,3,16.0,2019-01-01 01:00:00
324453,2019-01-01,4,16.0,2019-01-01 01:30:00
324454,2019-01-01,5,16.0,2019-01-01 02:00:00


In [197]:
SBP_.head()

,Date,Settlement_Period,System Buy Price(GBP/MWh),Time Interval,Time Format,Timestamp
0,2001-03-27,1,20.97549,0,00:00:00,2001-03-27 00:00:00
1,2001-03-27,2,19.58800,30,00:30:00,2001-03-27 00:30:00
2,2001-03-27,3,22.61021,60,01:00:00,2001-03-27 01:00:00
3,2001-03-27,4,38.08869,90,01:30:00,2001-03-27 01:30:00
4,2001-03-27,5,33.67798,120,02:00:00,2001-03-27 02:00:00


In [198]:
df = df.rename(columns={'OldColumnName': 'NewColumnName'})

SBP_ = SBP_.rename(columns= {'Timestamp':"DATETIME"})

In [199]:
SBP__ = SBP_.drop(['Time Interval','Time Format'],axis=1)

we have 4 datasets now demand data ,market index price ,generation data, SSP/SBP

#  Market Imbalance

## Market Imbalance Volume

##  https://data.nationalgrideso.com/balancing/daily-balancing-volume-balancing-services-use-of-system

In [219]:
import pandas as pd
import glob


# Define the selected filenames or patterns
selected_files_MI = ['q1_vol_fy1718.csv', 'q1_vol_fy1819.csv', 'q1_vol_fy1920.csv','q1_vol_fy2021.csv',
                  'q1_vol_fy2223.csv','q1_vol_fy2324.csv']


In [220]:
# Initialize an empty list to store the dataframes
dataframes = []

# Loop through the selected files
for file_pattern in selected_files_MI:
    df = pd.read_csv(file_pattern)
    dataframes.append(df)

# Concatenate all the DataFrames into a single DataFrame
Market_Imbalance = pd.concat(dataframes, ignore_index=True)




In [221]:
Market_Imbalance.head()

,SETT_DATE,SETT_PERIOD,Energy Imbalance (MWh),Frequency Control Offers (MWh),Frequency Control Bids (MWh),Positive Reserve (MWh),Constraint Offers (MWh),Constraint Bids (MWh),Negative Reserve (MWh),Other (MWh)
0,2017-04-01,1,-572.883,0.000,-516.195,0.0,325.0,-348.5,0.0,-33.188
1,2017-04-01,2,-559.190,25.000,-448.892,0.0,325.0,-348.5,0.0,-111.798
2,2017-04-01,3,-141.802,189.999,-312.807,0.0,325.0,-348.5,0.0,4.506
3,2017-04-01,4,40.125,260.850,-230.175,0.0,325.0,-348.5,0.0,32.950
4,2017-04-01,5,139.325,236.083,-209.191,0.0,325.0,-348.5,0.0,135.933


In [222]:
import datetime
#Demand_Data['date'] = Demand_Data['SETTLEMENT_DATE'].apply(lambda x: datetime.strptime(x, '%d %B %Y'))
Market_Imbalance['Time Interval'] = (Market_Imbalance['SETT_PERIOD'] - 1) * 30


Market_Imbalance['Time Format'] = Market_Imbalance['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())

Market_Imbalance['Time Format'] = Market_Imbalance['Time Format'].apply(lambda x: x.strftime('%H:%M:%S'))

Market_Imbalance['DATETIME'] = pd.to_datetime(Market_Imbalance['SETT_DATE'] + ' ' + Market_Imbalance['Time Format'])




In [218]:
Market_Imbalance.head()

,SETT_DATE,SETT_PERIOD,Energy Imbalance (MWh),Frequency Control Offers (MWh),Frequency Control Bids (MWh),Positive Reserve (MWh),Constraint Offers (MWh),Constraint Bids (MWh),Negative Reserve (MWh),Other (MWh),Time Interval,Time Format,DATETIME
0,2017-04-01,1,-572.883,0.000,-516.195,0.0,325.0,-348.5,0.0,-33.188,0,00:00:00,2017-04-01 00:00:00
1,2017-04-01,2,-559.190,25.000,-448.892,0.0,325.0,-348.5,0.0,-111.798,30,00:30:00,2017-04-01 00:30:00
2,2017-04-01,3,-141.802,189.999,-312.807,0.0,325.0,-348.5,0.0,4.506,60,01:00:00,2017-04-01 01:00:00
3,2017-04-01,4,40.125,260.850,-230.175,0.0,325.0,-348.5,0.0,32.950,90,01:30:00,2017-04-01 01:30:00
4,2017-04-01,5,139.325,236.083,-209.191,0.0,325.0,-348.5,0.0,135.933,120,02:00:00,2017-04-01 02:00:00


## Market imbalance cost

In [122]:
import pandas as pd
import glob


# Define the selected filenames or patterns
selected_files_MC = ['q1_cost_fy1718.csv', 'q1_cost_fy1819.csv', 'q1_cost_fy1920.csv','q1_cost_fy2021.csv',
                  'q1_cost_fy2223.csv','q1_cost_fy2324.csv']


In [123]:
dataframes = []

# Loop through the selected files
for file_pattern in selected_files_MC:
    df = pd.read_csv(file_pattern)
    dataframes.append(df)

# Concatenate all the DataFrames into a single DataFrame
Market_Cost = pd.concat(dataframes, ignore_index=True)




In [124]:
import datetime
#Demand_Data['date'] = Demand_Data['SETTLEMENT_DATE'].apply(lambda x: datetime.strptime(x, '%d %B %Y'))
Market_Cost['Time Interval'] = (Market_Cost['SETT_PERIOD'] - 1) * 30


Market_Cost['Time Format'] = Market_Cost['Time Interval'].apply(lambda x: (datetime.datetime.min + datetime.timedelta(minutes=x)).time())

Market_Cost['Time Format'] = Market_Cost['Time Format'].apply(lambda x: x.strftime('%H:%M:%S'))

Market_Cost['DATETIME'] = pd.to_datetime(Market_Cost['SETT_DATE'] + ' ' + Market_Cost['Time Format'])




In [225]:
Market_Cost.head()

,SETT_DATE,SETT_PERIOD,Energy Imbalance,Frequency Control,Positive Reserve,Negative Reserve,Constraints,Other,Time Interval,Time Format,DATETIME
0,01/04/2017,1,-16344.352,1547.211000,0.0,0.0,22737.95500,106.456000,0,00:00:00,2017-01-04 00:00:00
1,01/04/2017,2,-15959.283,2047.106000,0.0,0.0,22738.19000,395.865000,30,00:30:00,2017-01-04 00:30:00
2,01/04/2017,3,-4136.364,5780.480644,0.0,0.0,22909.11866,43.568692,60,01:00:00,2017-01-04 01:00:00
3,01/04/2017,4,1865.814,7260.858000,0.0,0.0,23160.25000,153.421000,90,01:30:00,2017-01-04 01:30:00
4,01/04/2017,5,6687.600,6359.364000,0.0,0.0,23195.50000,592.129000,120,02:00:00,2017-01-04 02:00:00


# Merging data 

In [58]:
Demand_Data.head()

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,Time Interval,Time Format,DATETIME
0,01-JAN-2009,1,37910,38704,33939,54,1403,0,0,0,...,0,0,-161,0,0,NaN,NaN,0,00:00:00,2009-01-01 00:00:00
1,01-JAN-2009,2,38047,38964,34072,53,1403,0,0,0,...,0,0,-160,0,0,NaN,NaN,30,00:30:00,2009-01-01 00:30:00
2,01-JAN-2009,3,37380,38651,33615,53,1403,0,0,0,...,0,0,-160,0,0,NaN,NaN,60,01:00:00,2009-01-01 01:00:00
3,01-JAN-2009,4,36426,37775,32526,50,1403,0,0,0,...,0,0,-160,0,0,NaN,NaN,90,01:30:00,2009-01-01 01:30:00
4,01-JAN-2009,5,35687,37298,31877,50,1403,0,0,0,...,0,0,-160,0,0,NaN,NaN,120,02:00:00,2009-01-01 02:00:00


In [62]:
Demand_Data['NSL_FLOW'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 253582 entries, 0 to 253581
Series name: NSL_FLOW
Non-Null Count  Dtype  
--------------  -----  
78286 non-null  float64
dtypes: float64(1)
memory usage: 1.9 MB


In [67]:
DD1 = Demand_Data.dropna().reset_index(drop =True)
# we are dropping few rows of years data we dont have data 

In [68]:
DD1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78286 entries, 0 to 78285
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   SETTLEMENT_DATE            78286 non-null  object        
 1   SETTLEMENT_PERIOD          78286 non-null  int64         
 2   ND                         78286 non-null  int64         
 3   TSD                        78286 non-null  int64         
 4   ENGLAND_WALES_DEMAND       78286 non-null  int64         
 5   EMBEDDED_WIND_GENERATION   78286 non-null  int64         
 6   EMBEDDED_WIND_CAPACITY     78286 non-null  int64         
 7   EMBEDDED_SOLAR_GENERATION  78286 non-null  int64         
 8   EMBEDDED_SOLAR_CAPACITY    78286 non-null  int64         
 9   NON_BM_STOR                78286 non-null  int64         
 10  PUMP_STORAGE_PUMPING       78286 non-null  int64         
 11  IFA_FLOW                   78286 non-null  int64         
 12  IFA2

In [69]:
DD1.head()

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,Time Interval,Time Format,DATETIME
0,01-JAN-2019,1,23808,25291,22393,2548,5918,0,13052,0,...,0,176,-455,-250,0,0.0,0.0,0,00:00:00,2019-01-01 00:00:00
1,01-JAN-2019,2,24402,25720,22962,2475,5918,0,13052,0,...,0,194,-455,-236,0,0.0,0.0,30,00:30:00,2019-01-01 00:30:00
2,01-JAN-2019,3,24147,25495,22689,2396,5918,0,13052,0,...,0,581,-410,-311,0,0.0,0.0,60,01:00:00,2019-01-01 01:00:00
3,01-JAN-2019,4,23197,24590,21849,2317,5918,0,13052,0,...,0,600,-450,-315,0,0.0,0.0,90,01:30:00,2019-01-01 01:30:00
4,01-JAN-2019,5,22316,24346,20979,2236,5918,0,13052,0,...,0,675,-442,-463,0,0.0,0.0,120,02:00:00,2019-01-01 02:00:00


In [76]:
DD1.columns

Index(['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD',
       'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION',
       'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION',
       'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING',
       'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW',
       'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'Time Interval',
       'Time Format', 'DATETIME'],
      dtype='object')

In [75]:
len(DD1.columns)

22

In [78]:
len(gen_data.columns)

32

In [216]:
gen_data.head();

In [82]:
merg1 = pd.merge(DD1, gen_data, on='DATETIME', how='inner') 
#demand data and generation data are merged

In [88]:
merg2 = pd.merge(merg1, mid_1, on='DATETIME', how='inner')
#market index price is merged to previous data

In [130]:
SSP__.head()

,Date,Settlement_Period,System Sell Price(GBP/MWh),DATETIME
0,2001-03-27,1,2.76676,2001-03-27 00:00:00
1,2001-03-27,2,6.23181,2001-03-27 00:30:00
2,2001-03-27,3,5.92955,2001-03-27 01:00:00
3,2001-03-27,4,0.61020,2001-03-27 01:30:00
4,2001-03-27,5,0.12778,2001-03-27 02:00:00


In [212]:
SSP__[SSP__['Date'] == '2019-01-01'];

In [203]:
merg3 = pd.merge(merg2,SSP__, on = 'DATETIME', how = 'inner')

In [211]:
# after 2015 SBP and SSP are same 

In [215]:
merg4 = pd.merge(merg3,Market_Cost, on = 'DATETIME', how = 'inner')

In [223]:
merg5 = pd.merge(merg3,Market_Imbalance, on = 'DATETIME', how = 'inner')

In [229]:
merg5.columns

Index(['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD',
       'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION',
       'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION',
       'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING',
       'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW',
       'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'Time Interval_x',
       'Time Format_x', 'DATETIME', 'GAS', 'COAL', 'NUCLEAR', 'WIND', 'HYDRO',
       'IMPORTS', 'BIOMASS', 'OTHER', 'SOLAR', 'STORAGE', 'GENERATION',
       'CARBON_INTENSITY', 'LOW_CARBON', 'ZERO_CARBON', 'RENEWABLE', 'FOSSIL',
       'GAS_perc', 'COAL_perc', 'NUCLEAR_perc', 'WIND_perc', 'HYDRO_perc',
       'IMPORTS_perc', 'BIOMASS_perc', 'OTHER_perc', 'SOLAR_perc',
       'STORAGE_perc', 'GENERATION_perc', 'LOW_CARBON_perc',
       'ZERO_CARBON_perc', 'RENEWABLE_perc', 'FOSSIL_perc',
       'Settlement Period', 'Market Index Price (£/MWh)',
       'Market Index Volume (MWh)', 'Date', 'Se

In [227]:
drop1 = merg5.drop(['Time Interval_x','Time Format_x','Time Interval_y','Time Format_y'],axis=1)

In [230]:
drop1.columns

Index(['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD',
       'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION',
       'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION',
       'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING',
       'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW',
       'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'DATETIME', 'GAS', 'COAL',
       'NUCLEAR', 'WIND', 'HYDRO', 'IMPORTS', 'BIOMASS', 'OTHER', 'SOLAR',
       'STORAGE', 'GENERATION', 'CARBON_INTENSITY', 'LOW_CARBON',
       'ZERO_CARBON', 'RENEWABLE', 'FOSSIL', 'GAS_perc', 'COAL_perc',
       'NUCLEAR_perc', 'WIND_perc', 'HYDRO_perc', 'IMPORTS_perc',
       'BIOMASS_perc', 'OTHER_perc', 'SOLAR_perc', 'STORAGE_perc',
       'GENERATION_perc', 'LOW_CARBON_perc', 'ZERO_CARBON_perc',
       'RENEWABLE_perc', 'FOSSIL_perc', 'Settlement Period',
       'Market Index Price (£/MWh)', 'Market Index Volume (MWh)', 'Date',
       'Settlement_Period', 'System Sell Price